## Previsão de insatisfação de cliente em compras de e-commerce a fim de permitir ações proativas de forma preventiva
 - Projeto de Disciplina - Redes neurais com TensorFlow [25E3_2]
 - Aluna: Rosana Ribeiro Lima
 - Dataset: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_orders_dataset.csv


-------------------------------
### Pitch de negócio

##### Problema:

Com frequência, empresas de e-commerce perdem a fidelização de clientes em razão de instatisfação, seja por atrasos, falhas na entrega, e outros motivos. Isso gera inevitavelmente perdas na receita da empresa, além de impactar a imagem que as pessoas tem sobre a confiabilidade da instituição. 

Saber prever a insatisfação do cliente (que pode ser identificada por avaliações negativas) com base em dados do pedido antes da entrega ser finalizada poderia permitir ações proativas, como envio de brindes pelos vendedores, antecipação de problemas ou melhoria logística, no caso de atrasos.

##### Solução proposta:

A solução proposta visa analisar o dataset em questão e utilizá-lo como base para criar um modelo de redes neurais que permita identificar as principais causas que possam ser identificadas previamente como motivo para eventuais avaliações ruins de um pedido.

Além disso, caso seja identificado que o atraso na entrega tem um peso importante na avaliação ruim do pedido, a solução se propõe a identificar eventuais features que possam impactar nesse atraso, a fim de que medidas preventivas de logísticas possam ser tomadas pela empresa.

--------------------
### Features propostas

Com base no cruzamento de dados das tabelas disponíveis no dataset em questão, as features propostas para utilização são:

- payment_type
- payment_value
- estado_do_comprador
- estado_do_vendedor
- tamanho_da_descricao_do_produto (número de caracteres)
- categoria_do_produto
- número_de_fotos_do_produto
- peso_do_produto
- altura_do_produto
- dia da compra
- tempo da compra até a aprovacao do pagamento
- tempo da confirmacao da compra até a entrega
- diferenca entre tempo previsto para a entrega e tempo real de entrega
- quantidade de itens no pedido


In [1]:
import os
import pandas as pd

path = "../data/"
# df = pd.read_csv(os.path.join(path, "synthetic_asthma_dataset.csv"))
# df.head(20)

### 2. Descrição e análise das variáveis - **COMPREENSÃO DOS DADOS**


#### 2.1 Descrevendo as variáveis e os tipos

Vamos primeiro fazer algumas análises dos dados:

In [2]:
# df.info()

In [3]:
# df_filtered['Asthma_Status'].value_counts()

## Referências

- OLIST. Brazilian E-Commerce Public Dataset by Olist. Kaggle, 2018. Disponível em: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce. Acesso em: 31 jul. 2025.